In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from src import SSD, AnchorGenerator, FeatureExtractor
from src.backbones import mobilenet_v1_base
from src.anchor_generator import tile_anchors

# Generate anchors

In [ ]:
tf.reset_default_graph()

height, width = 360, 640
images = tf.placeholder(tf.float32, [None, 3, height, width])

is_training = True
def backbone(images, is_training):
    return mobilenet_v1_base(images, is_training, min_depth=8, depth_multiplier=1.0)
    
feature_extractor = FeatureExtractor(backbone, is_training)
anchor_generator = AnchorGenerator()

feature_maps = feature_extractor(images)
anchors = anchor_generator(feature_maps, images)


more_anchors = tile_anchors(
    width/height,
    6, 10, scales=(0.5, 0.5, 0.5, 0.5, 0.5, 0.57), 
    aspect_ratios=(1.0, 2.0, 3.0, 0.5, 0.333, 1.0), 
    anchor_stride=(1/6, 1/10), 
    anchor_offset=(1/12, 1/20)
)

In [ ]:
feature_maps

In [ ]:
anchors

In [ ]:
anchor_generator.num_basis_anchors

In [ ]:
num_anchors_per_feature_map = anchor_generator.num_anchors_per_feature_map
num_anchors_per_feature_map

In [ ]:
anchor_generator.feature_map_shape_list

In [ ]:
with tf.Session() as sess:
    anchor_boxes = sess.run(anchors)
    more_anchor_boxes = sess.run(more_anchors)
    

i = 2
start = sum(num_anchors_per_feature_map[:i])
anchor_boxes = anchor_boxes[start:(start + num_anchors_per_feature_map[i])]
anchor_boxes.shape

In [ ]:
more_anchor_boxes.shape

In [ ]:
# anchor_boxes = anchor_boxes.reshape((6, 10, 6, 4))
more_anchor_boxes = more_anchor_boxes.reshape((6, 10, 6, 4))

# Show non clipped anchors

In [ ]:
ymin, xmin, ymax, xmax = [more_anchor_boxes[:, :, :, i] for i in range(4)]

h, w = height*(ymax - ymin), width*(xmax - xmin)
cy, cx = height*ymin + 0.5*h, width*xmin + 0.5*w

centers = np.stack([cy, cx], axis=3)
anchor_sizes = np.stack([h, w], axis=3)

In [ ]:
fig, ax = plt.subplots(1, dpi=100, figsize=(int(5*width/height), 5))
unique_centers = centers[:, :, 0, :].reshape(-1, 2)
unique_sizes = anchor_sizes[0, 0, :, :]

i = 25
for j, point in enumerate(unique_centers):
    cy, cx = point
    color = 'g' if j == i else 'r' 
    ax.plot([cx], [cy], marker='o', markersize=3, color=color)

cy, cx = unique_centers[i] 
for box in unique_sizes:
    h, w = box
    xmin, ymin = cx - 0.5*w, cy - 0.5*h
    rect = plt.Rectangle(
        (xmin, ymin), w, h,
        linewidth=1.0, edgecolor='k', facecolor='none'
    )
    ax.add_patch(rect)

plt.xlim([0, width]);
plt.ylim([0, height]);